In [ ]:
import json
import itertools
import pickle
import hickle 
import gzip
import operator
import os
import sys
from time import time
import pprint as pp
import collections

import numpy as np
import pandas as pd

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cross_validation import train_test_split
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.externals import joblib


# bokeh
import bokeh.plotting as bkplt


# import requirments 
from IPython.display import Image
from IPython.display import display
import matplotlib.pyplot as plt
import json
import rpy2
%load_ext rpy2.ipython
%R require("ggplot2")
% matplotlib inline
from ggplot import *
randn = np.random.randn



# Topic Modeling
The first step was to remove retweets so the topics were not obscured by repeated tweets.

The second stop was to make use of the NLP enrichments. The parts of speech in the NLP enrichment allowed us to __extract the nouns__. This extraction process is detailed in `organize_data.py`.

In summary, retweets were removed and nouns were used in the vocabulary for the topic model.

### Filter noun set
Notice that we have many nouns (exact count above) that are potential features. We'll need to refine this number of features to focus on those dimensions that are most meaningful. 

At least two options exist:
1.  We can create an ordered list of the nouns frequencies, graph them, and then remove the values outside of our desired thresholds. 
2.  We can create a list of tweets that contain only nouns, vectorize them with TFIDF (and add bigrams). 

We'll play with option #1 and then focus on option #2. 



In [ ]:
# organize nouns counts from largest to smallest
sorted_counts = sorted(counts.items(), key=operator.itemgetter(1), reverse=True)

# convert data to pandas df
df = pd.DataFrame(sorted_counts, columns=('noun','counts'))

# build the initial count for the summing process (we'll build a cdf)
cdf = sorted_counts[0][1]

# total of all counts
total = df.counts.sum()

# list w/ initial element [(term, count, summed_count, percentile, delta)]
cdf_data = [(sorted_counts[0][0]
             , sorted_counts[0][1]
             , sorted_counts[0][1]
             , 1-(sorted_counts[0][1]*1.0/total)
             , 0 )]

# sum up totals (start at 1; do not start at 0)
for i in range(1,len(sorted_counts)):
    prev_cdf = cdf
    cdf += sorted_counts[i][1]
    # list containing cdf info [(term, count, summed_count, percentile, delta)]
    cdf_data.append((sorted_counts[i][0]
                     , sorted_counts[i][1]
                     , cdf
                     , 1- (cdf*1.0/total)
                     , cdf-prev_cdf ))

# view cdf results
display(cdf_data[:20])

# create dataframe from the list cdf_data
cdf_data_df = pd.DataFrame(cdf_data,columns=('noun','count','sum','percentile', 'delta'))

# bucket counts for specific percentiles defined in the range
pL = []
percentile = []
for i in range(50,100,5):
    pL.append(len(cdf_data_df[cdf_data_df.percentile>(1-(i/100.0))]))
    percentile.append(str(i))


# create dataframe for plotting
plot_data = pd.DataFrame(
    {'sumcounts': pL
     , 'percentiles': percentile
    }
)

# show total counts
display(plot_data.head())
print 'Total nouns: {:,}'.format(len(counts))

# send the result to R
%Rpush plot_data 

Exploring the cumulative distribution function, we can consider the frequency of the nouns and reduce the current 677K dimensions. The plot below suggests that the majority of the noun counts are above the 85th percentile. Exploring this data a little, we see that the few nouns in the upper percentils contain the majority of the counts.

In [ ]:
%%R

#ggplot(data=plot_40, aes(x=percentile,y=sum))+geom_bar(stat="identity")+ggtitle("CDF: Noun Counts")
ggplot(data=plot_data, aes(x=percentiles,y=sumcounts))+geom_bar(stat="identity")+coord_flip()+ggtitle("CDF: Noun Counts")+xlab('Percentile (What % of nouns are below this set?)')+ylab('Set Size (Counts associated w/ Nouns in this set.)')
#head(plot_50)
#summary(plot_50)

So under option #1, we can use the graph above to inform a method to restrict the features in our dataset. 

In [ ]:
reduced_vocab = cdf_data_df[cdf_data_df.percentile>0.40]
display(reduced_vocab.head())
print 'Nouns to use in Option1 vocabulary: {:,}'.format(len(reduced_vocab))

In [ ]:
# write reduced vocab
reduced_vocab.to_pickle('reduced_vocab.pkl')

### Build Vectorizers 
Option1: We'll use the newly reduced vocabulary to build a vectorizer. (previously completed)  
Option2: We'll use the vectorizer's TFIDF utility to build a vocabulary set. (completed below)

In [ ]:
# see http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

def vectorize_1(vocab):
    vectorizer = TfidfVectorizer(#min_df=20
                                 stop_words='english'
                                 , use_idf=True # enable inverse-document-frequency reweighting
                                 , ngram_range=(1,2) # given our vocab, not really necessary
                                 , binary = True # presence of word instead of frequency
                                 , vocabulary = vocab
                                ) 
    #X = vectorizer.fit_transform(tweet_list)
    return vectorizer

def vectorize_2():
    vectorizer = CountVectorizer(min_df=20
                                 , stop_words='english'
                                 , ngram_range=(1,2) # given our vocab, not really necessary
                                 , binary = True # presence of word instead of frequency
                                 #, vocabulary = set(vocab)
                                ) 
    #X = vectorizer.fit_transform(tweet_list)
    return vectorizer


We build the vectorizers below.

In [ ]:
# Build the vectorizer for option1
red_vectorizer = vectorize_1(reduced_vocab.noun)
# Build the vectorizer for option2
noun_vectorizer = vectorize_2()

### Testing/Training Data

We'll apply the vectorizer function that we built earlier, form the doc term matrix on the training set, and develop the topic model. 

In [ ]:
run_here = False

def create_index(total_tweets):
        """
        Builds an index for the training and test set.
        The sets serve as a list of row numbers to extract from the dataset. 
        """
        # based on the total tweet count, create an array of all line numbers 
        line_index = np.array(range(0,total_tweets))
        line_index_nouns = np.array(range(0,len(all_tweets_nouns_only)))
        # split the array into training and test sets of index values
        trainIndex,testIndex = train_test_split(line_index,train_size=0.20, random_state=42)
        trainIndex_nouns,testIndex_nouns = train_test_split(line_index_nouns,train_size=0.20, random_state=42)
        # save test & traning index values
        np.save("training_index",trainIndex)
        np.save("testing_index",testIndex)
        return trainIndex,testIndex, trainIndex_nouns,testIndex_nouns

if run_here:
    trainIndex,testIndex, trainIndex_nouns,testIndex_nouns = create_index(total_tweets)

We can now get the text for the training and test sets. 

In [ ]:
def get_txt_data(index_set, infile, out_file_path='_data.pkl',write_to_file=False, json_format=False):
    """
    Reads the infile and uses the appropriate index to return the training/test data.
    """
    if isinstance(infile,list):
        txt = [infile[i] for i in index_set]
    elif not json:
        with gzip.open(infile,'rb') as body_file:
            # read all lines into memory and form a list
            body_list = body_file.readlines()
            txt = [body_list[i] for i in index_set]
    else:
        with gzip.open(infile,'rb') as body_file:
            # read each line, parse json, and form a list
            txt = []
            index_lookup = dict([(i,0) for i in index_set])
            for i,line in enumerate(body_file):
                #print line
                #print (i not in index_lookup)
                if i not in index_lookup:
                    continue
                rec = json.loads(line)
                txt.append(rec['body']) 
            
    if write_to_file:
        with open(out_file_path,'wb') as f:
            # save txt
            pickle.dump(txt, f)
            
    return txt



We'll build the traning/test sets for each of the models below.

In [ ]:
###
### Build model specific traning sets
###
run_here = False 
if run_here:
    training_txt_list = get_txt_data(
        trainIndex
        , '<filePath>'
        , 'training_txt.pkl'
        , True
        , True)

    testing_txt_list = get_txt_data(
        testIndex
        , '<filePath>'
        , 'testing_txt.pkl'
        , True
        , True)

    training_noun_list = get_txt_data(
        trainIndex_nouns
        , all_tweets_nouns_only
        , 'training_nouns_txt.pkl'
        , True
        , True)

    testing_noun_list = get_txt_data(
        testIndex_nouns
        , all_tweets_nouns_only
        , 'testing_nouns_txt.pkl'
        , True
        , True)


Saving this data to disk, we won't need to run this section each time we open the notebook.

In [ ]:
run_here = False 
if run_here:
    pickle.dump(training_txt_list,open('training_txt_list','wb'))
    pickle.dump(testing_txt_list,open('testing_txt_list','wb'))
    pickle.dump(training_noun_list,open('training_noun_list','wb'))
    pickle.dump(testing_noun_list,open('testing_noun_list','wb'))

In [ ]:
run_here = True
if run_here:
    training_txt_list = pickle.load(open('training_txt_list','rb'))
    testing_txt_list = pickle.load(open('testing_txt_list','rb'))
    training_noun_list = pickle.load(open('training_noun_list','rb'))
    testing_noun_list = pickle.load(open('testing_noun_list','rb'))

In [ ]:
print "manual reduction method"
print "  len(training_txt_list): {:,} | len(testing_txt_list): {:,}.".format(len(training_txt_list),len(testing_txt_list))
print "Nouns w/ bigrams method:"
print "  len(training_txt_list): {:,} | len(testing_txt_list): {:,}.".format(len(training_noun_list),len(testing_noun_list))

### Vectorize the data
We'll now apply our vectorizers to the training data.

In [ ]:
# Vectorize the tweets for option1 
X_reduced = red_vectorizer.fit_transform(training_txt_list)

# Vectorize the tweets for option2 
X_noun = noun_vectorizer.fit_transform(training_noun_list)



In [ ]:
pickle.dump(noun_vectorizer.get_feature_names(),open('noun_vectorizer_vocab','wb'))

In [ ]:
print "total vocab manual reduction: {:,} | total vocab TFIDF reduction w/ bigrams: {:,}".format(len(red_vectorizer.get_feature_names()),len(noun_vectorizer.get_feature_names()))

### Dimension reduction?
We'll apply SVD to reduce the number of dimensions in a new space. We use kmeans to build 5 centroids in the data. These centroids will serve as the basis for our topic clusters. 

The first step is to decide how many components to use for SVD.

### Percent of explained variance
Reducing the numbrer of components affects the amount of variance that we can explain. The graph below helped us determine a reasonable number of compnents to use when applying the topic model. Notice that almost 70% of the variance is explained using 300 components for the manual reduction but we need over 1000 components to explain the same amount of variance in the noun based model.

In [ ]:
#explained_variances = np.var(X_svd, axis=0) / np.var(X_train, axis=0).sum()

### 
### This can take 10-15 mins
###
def create_svd_doc_term_matrix(X_train, num_eigen_vectors=100):
    """
    Create the array with truncated svd.
    """
    svd = TruncatedSVD(n_components = num_eigen_vectors)
    lsa = make_pipeline(svd, Normalizer(copy=False))
    return lsa.fit_transform(X_train), svd

run_here = False
if run_here:
    explained_variance_red = []
    explained_variance_nouns = []
    svd_component_range = range(100,351,50)
    for i in svd_component_range:
        # find explained variance in manual reduction method
        X_svd_red, svd_red = create_svd_doc_term_matrix(X_reduced,i)
        explained_variance_red.append(svd_red.explained_variance_ratio_.sum())
        # find explained variance in noun reduction method
        X_svd_noun, svd_noun = create_svd_doc_term_matrix(X_noun,i*3)
        explained_variance_nouns.append(svd_noun.explained_variance_ratio_.sum())



From the data below, the idea is that we will use only 300 components for the manual reduction model but 1,050 components for the noun model.

In [ ]:
run_here = False
if run_here:
    expVar = pd.DataFrame({'explained_red':explained_variance_red
                       , 'explained_nouns':explained_variance_nouns
                       , 'components_red':svd_component_range
                       , 'components_nouns':[comp*3 for comp in svd_component_range] })
    display(expVar)
    #%Rpush  expVar
    display(expVar.plot(x='components_red',y='explained_red'))
    display(expVar.plot(x='components_nouns',y='explained_nouns'))
    #list(expVar.components)

With SVD, we can __reduced__ the number of our dimensions even further. In the next section, we'll use this technique and build the cluster centroids. 

In [ ]:
#p = bkplt.figure(title="Explained Variance per n Components"
#                 , x_axis_label='N Components'
#                 , y_axis_label='Explained Variance')
#p.circle(list(expVar.components), list(expVar.explained), legend="Variance Explained", fill_color="red", line_color="red", size=6)
#bkplt.show(p)

In [ ]:
#%%R
#print(expVar)
#ggplot(data=expVar)+geom_bar(aes(x=components,y=explained))

### Create cluster centroids
We'll now apply kmeans to find the centroids that will be used to predict a cluster for each tweet.

In [ ]:
def build_clusters(X_svd, k=5):
    """
    Use kmeans to find centroids.
    """
    km = KMeans(n_clusters=k
                , init='k-means++'
                , max_iter=100
                #, n_init=10
                , verbose=False)
    km.fit(X_svd)
    pred=km.predict(X_svd)
    pred_df=pd.DataFrame(pred)
    pred_df.columns=['pred_cluster']
    return km.cluster_centers_ , pred_df, k



In [ ]:
run_here = False
if run_here:
    centroids_noun, predictions_noun, n_clusters_noun = build_clusters(X_svd_noun, 5)
    centroids_red, predictions_red, n_clusters_red = build_clusters(X_svd_red, 5)
    centroids_noun50, predictions_noun50, n_clusters_noun50 = build_clusters(X_svd_noun, 50)

In [ ]:
# save data
run_here = False
if run_here:
    pickle.dump(centroids_noun, open('centroids_noun','wb'))
    pickle.dump(predictions_noun, open('predictions_noun','wb'))

    pickle.dump(centroids_red, open('centroids_red','wb'))
    pickle.dump(predictions_red, open('predictions_red','wb'))

    pickle.dump(centroids_noun50, open('centroids_noun50','wb'))
    pickle.dump(predictions_noun50, open('predictions_noun50','wb'))



In [ ]:
centroids_noun = pickle.load(open('centroids_noun','rb'))
predictions_noun = pickle.load(open('predictions_noun','rb'))

centroids_red = pickle.load(open('centroids_red','rb'))
predictions_red = pickle.load(open('predictions_red','rb'))

centroids_noun50 = pickle.load(open('centroids_noun50','rb'))
predictions_noun50 = pickle.load(open('predictions_noun50','rb'))

### Interpret the clusters as topics?
We can look at the top words loaded onto each cluster to consider a human readible forms. To understand the vocabulary used in the svd space, we need to transform from svd space to the original dimensions, which also provides the word loadings.

In [ ]:
run_here = False
if run_here:
    word_loadings_red=np.dot(centroids_red, svd_red.components_)
    pickle.dump(word_loadings_red, open('word_loadings_red','wb'))

    word_loadings_noun=np.dot(centroids_noun, svd_noun.components_)
    pickle.dump(word_loadings_noun, open('word_loadings_noun','wb'))

    word_loadings_noun50=np.dot(centroids_noun50, svd_noun.components_)
    pickle.dump(word_loadings_noun50, open('word_loadings_noun50','wb'))

In [ ]:
word_loadings_red=pickle.load(open('word_loadings_red','rb'))
#np.save('word_loadings_red',word_loadings_red)
#word_loadings_red = np.load('./word_loadings_red.npy')
#word_loadings=np.dot(centroids, svd.components_)
#print(word_loadings.shape)
#vocab=vectorizer.get_feature_names()
#for k in range(0,n_clusters):
run_here = False
if run_here:
    for k in range(0,5):
        #word loadings = cluster_centers * eigenvectors 
        indices=[i for i in np.argsort(word_loadings_red[k,:])[::-1]]    
        sorted_vocab=[reduced_vocab.noun[i] for i in indices]
        print("Top words for cluster {}:\n{}\n".format(k, sorted_vocab[:50]))

In [ ]:
word_loadings_noun = pickle.load(open('word_loadings_noun','wb'))

#np.save('word_loadings_noun',word_loadings_noun)
#word_loadings_noun = np.load('./word_loadings_noun.npy')
#word_loadings=np.dot(centroids, svd.components_)
#print(word_loadings.shape)
#vocab=vectorizer.get_feature_names()
#for k in range(0,n_clusters):
run_here = False
if run_here:
    for k in range(0,5):
        #word loadings = cluster_centers * eigenvectors 
        indices=[i for i in np.argsort(word_loadings_noun[k,:])[::-1]]    
        sorted_vocab=[noun_vectorizer.get_feature_names()[i] for i in indices]
        print("Top words for cluster {}:\n{}\n".format(k, sorted_vocab[:50]))
    
    

In [ ]:
word_loadings_noun50 = pickle.load(open('word_loadings_noun50','wb'))
#np.save('word_loadings_noun50',word_loadings_noun50)
#word_loadings_noun = np.load('./word_loadings_noun.npy')
#word_loadings=np.dot(centroids, svd.components_)
#print(word_loadings.shape)
#vocab=vectorizer.get_feature_names()
#for k in range(0,n_clusters):
run_here = False
if run_here:  
    for k in range(0,50):
        #word loadings = cluster_centers * eigenvectors 
        indices=[i for i in np.argsort(word_loadings_noun50[k,:])[::-1]]    
        sorted_vocab=[noun_vectorizer.get_feature_names()[i] for i in indices]
        print("Top words for cluster {}:\n{}\n".format(k, sorted_vocab[:50]))

### Test the model
We'll now use these cluster centers to consider the test tweets. 

In [ ]:
def label_tweets(vectorizer, word_loadings, testing_data, sample_percentage=0.20):
    """
    Label tweets.
    """
    result = []
    sample_size = int(len(testing_data)*sample_percentage)
    sample_tweets = testing_data[:sample_size]
    for tweet in sample_tweets:
        # vectorize the tweet
        sparse_array = vectorizer.fit_transform([tweet])
        # subtract all values between the tweet vectorization and centroids
        sparse_array_subtraction_abs = np.absolute(sparse_array - word_loadings)
        # sum to get the total distances 
        sparse_array_subtraction_abs_sum = sparse_array_subtraction_abs.sum(axis=1)
        # append the index of the minimum distance
        result.append(np.argmin(sparse_array_subtraction_abs_sum))
    return result



In [ ]:
type(vocab)

In [ ]:
def label_tweets(vectorized_tweet, word_loadings):
    """
    Label tweets.
    """
    result = []
    # vectorize the tweet
    sparse_array = vectorized_tweet
    # subtract all values between the tweet vectorization and centroids
    sparse_array_subtraction_abs = np.absolute(sparse_array - word_loadings)
    # sum to get the total distances 
    sparse_array_subtraction_abs_sum = sparse_array_subtraction_abs.sum(axis=1)
    # append the index of the minimum distance
    result.append(np.argmin(sparse_array_subtraction_abs_sum))
    myArray = sparse_array_subtraction_abs_sum
    return result,myArray

def vectorize_data(tweet_list,custom_vocab=False, vocab=[]):
    """
    Use the TFidfVectorizer or CountVectorizer to vectorize a set of tweets. 
    """
    sys.stdout.write('Vectorizing the tweets.'+'\n')
    #vectorizer = TfidfVectorizer(min_df=20
    vectorizer = CountVectorizer(#min_df=20
                                 #, use_idf = True
                                 stop_words='english'
                                 , ngram_range=(1,2)
                                 , binary = True # presence of word instead of frequency
                                 , vocabulary = vocab
                                ) 
    X = vectorizer.fit_transform(tweet_list)
    if len(vocab)==0:
        vocab = vectorizer.get_feature_names()
    sys.stdout.write('  - len(vocab):{:,}\n'.format(len(vocab)))
    return X

craft45_word_loadings = hickle.load('../enrichments/models/word_loadings50_bigrams_v1.hkl')
craft45_vocab = pickle.load(open('../enrichments/models/vocab_bigrams_v1.pkl'))

label,myArray = label_tweets(vectorize_data(['I like beer'],True, craft45_vocab),craft45_word_loadings)


In [ ]:
display(np.min(myArray), np.std(myArray), np.max(myArray), np.mean(myArray))
display(myArray)

In [ ]:
red_result = label_tweets(red_vectorizer, word_loadings_red, testing_txt_list)

In [ ]:
pickle.dump(red_result,open('red_result','wb'))

In [ ]:
noun_vocab = pickle.load(open('noun_vectorizer_vocab','rb'))
noun_vectorizer.set_params(min_df = 0.1, vocabulary = noun_vocab)
noun_result = label_tweets(noun_vectorizer, word_loadings_noun, testing_noun_list)
pickle.dump(noun_result,open('noun_result','wb'))

In [ ]:
noun_vectorizer.set_params(min_df = 0.1, vocabulary = noun_vocab)
noun50_result = label_tweets(noun_vectorizer, word_loadings_noun50, testing_noun_list)
pickle.dump(noun50_result,open('noun_result50','wb'))

# Iterate (shorter cycle)
Run the entire process in a single function.

In [ ]:
! gzip -cd /Users/blehman/... | wc -l

In [ ]:
tweet_body = '/Users/blehman/...'
tweet_body = '/Users/blehman/...'
with gzip.open(tweet_body, 'rb') as tweet_set:
    print len(tweet_set.readlines())

In [ ]:
tweet_body = '/Users/blehman...'
sum([1 for line in gzip.open(tweet_body, 'rb')])

In [ ]:
import os
tweet_body = '/Users/blehman/...'
os.path.getsize(tweet_body)

In [ ]:
with open ('test','wb') as f:
    x = [1,2,3]
    for line in x:
        f.write(str(line)+'\n')
print os.path.getsize('test')
with open ('test','rb') as f:
    print f.readlines()

In [ ]:
def create_index(tweet_list, set_name, training_percent=0.20,write_to_file=False):
        """
        Builds an index for the training and test set.
        The sets serve as a list of row numbers to extract from the dataset. 
        """
        # get file len
        file_len = len(tweet_list)
        # based on the total tweet count, create an array of all line numbers 
        file_len_array = np.array(range(0,file_len))
        
        # split the array into training and test sets of index values
        sys.stdout.write('Build training/test sets.' + '\n')
        train_index,test_index = train_test_split(file_len_array,train_size=training_percent, random_state=42)
        
        # save test & traning index values
        if write_to_file:
            sys.stdout.write('  - writing training/test sets to file.' + '\n')
            hickle.dump(train_index, "training_index_"+set_name + ".hkl", mode='w', compression='gzip')
            hickle.dump(test_index, "testing_index_"+set_name + ".hkl", mode='w', compression='gzip')
        print train_index
        return train_index,test_index

def vectorize_data(tweet_list,custom_vocab=False, vocab=[]):
    """
    Use the TFidfVectorizer or CountVectorizer to vectorize a set of tweets. 
    """
    sys.stdout.write('Vectorizing the tweets.'+'\n')
    #vectorizer = TfidfVectorizer(min_df=20
    vectorizer = CountVectorizer(min_df=20
                                 #, use_idf = True
                                 , stop_words='english'
                                 , ngram_range=(1,2)
                                 , binary = True # presence of word instead of frequency
                                 #, vocabulary = set(vocab)
                                ) 
    X = vectorizer.fit_transform(tweet_list)
    if len(vocab)==0:
        vocab = vectorizer.get_feature_names()
    sys.stdout.write('  - len(vocab):{:,}\n'.format(len(vocab)))
    return X, vocab



def get_data(tweet_list,index_):
    sys.stdout.write('Getting specific tweets for training/test.' + '\n')
    data = [tweet_list[i] for i in index_]
    return data
    

def create_svd_doc_term_matrix(X_train, num_eigen_vectors=100):
    """
    Create the array with truncated svd.
    """
    sys.stdout.write('Using SVD w/ {:,} components'.format(num_eigen_vectors) + '\n')
    svd = TruncatedSVD(n_components = num_eigen_vectors)
    pipeline = make_pipeline(svd, Normalizer(copy=False))
    return pipeline.fit_transform(X_train), svd

def view_explained_variance(X_train,svd_component_range = range(100,351,50)):
    """
    Builds graphs to determine an appropriate number of SVD components to use w/ kmeans.
    """
    explained_variance = []

    # build list of explained variance values
    sys.stdout.write('Testing w/ {} components.'.format(svd_component_range) + '\n')
    for i in svd_component_range:
        # find explained variance 
        X_svd, svd = create_svd_doc_term_matrix(X_train,i)
        exp_var = svd.explained_variance_ratio_.sum()
        print "  - Explained variance w/ {} components: {}".format(i,exp_var)
        explained_variance.append(exp_var)
    
    # graph values
    expVar = pd.DataFrame({'components':svd_component_range
                           ,'explained_variance':explained_variance
                          })
    display(expVar)
    #%Rpush  expVar
    display(expVar.plot(x='components',y='explained_variance'))
    #list(expVar.components)


def build_clusters(X_svd, k=5):
    """
    Use kmeans to find centroids.
    """
    sys.stdout.write('Building clusters w/ k={}.'.format(k) + '\n')
    km = KMeans(n_clusters=k
                , init='k-means++'
                , max_iter=100
                #, n_init=10
                , verbose=False)
    km.fit(X_svd)
    pred=km.predict(X_svd)
    pred_df=pd.DataFrame(pred)
    pred_df.columns=['pred_cluster']
    return km.cluster_centers_ , pred_df


In [ ]:
# config 
gziped_tweet_bodies = '/Users/blehman/...'
training_set_size = 0.99 # percentage of data to use as training to build centroids
load_prev_saved_training_set = False
tweet_set_label = '_50cluster'
num_clusters = 50

for iteration in range(0,3):
    # run process
    with gzip.open(gziped_tweet_bodies, 'rb') as tweet_file_obj:
        tweet_list = tweet_file_obj.readlines()
        sys.stdout.write('')
        if load_prev_saved_training_set:
            # load train/test index values
            train_index = hickle.load("training_index_" + tweet_set_label + ".hkl")
            test_index = hickle.load("testing_index_" + tweet_set_label + ".hkl")
        else: 
            # build train/test index values
            train_index, test_index = create_index(tweet_list, tweet_set_label ,training_set_size, True)

        # get specific tweets 
        X_train_txt_list = get_data(tweet_list, train_index)


    #sys.stdout.write("len(X_train_txt_list):{:,} \n".format(len(X_train_txt_list)))
    display(X_train_txt_list[:10])


    # vectorize data
    X_train_vect, vocab = vectorize_data(X_train_txt_list)

    #(note: this step's runtime is 10-20 mins and can be skipped)
    #view explained variance to select number of components to use w/ SVD 
    #view_explained_variance(X_train_vect,[3*x for x in range(100,351,50)])

    # reduce dimensions with SVD
    num_components = 600
    X_train_svd, svd = create_svd_doc_term_matrix(X_train_vect, num_components)

    # build clusters (note: this can take over 30 mins)
    centroids, predictions = build_clusters(X_train_svd, num_clusters)

    # save centroids and vocab
    sys.stdout.write('Clusters Built. Saving data...'+'\n')
    hickle.dump(centroids, "centroids5_"+tweet_set_label + "_bigrams_v" + str(iteration) +".hkl", mode='w', compression='gzip')
    
    #hickle.dump(vocab, "vocab_"+tweet_set_label + ".hkl", mode='w', compression='gzip')
    pickle.dump(vocab, open("vocab_"+tweet_set_label + "_bigrams_vb" + str(iteration) + ".pkl",'wb'), protocol=2) 
    #pickle.dump(centroids, open("centroids_"+tweet_set_label + ".pkl",'wb'), protocol=2)
    word_loadings = np.dot(centroids, svd.components_)
    hickle.dump(word_loadings, "word_loadings" + str(num_clusters)+ "_" + tweet_set_label + "_bigrams_vb" + str(iteration) + ".hkl", mode='w', compression='gzip')
    
    sys.stdout.write("Printing wordloadings {} to file. \n".format(word_loadings.shape))
    with open("top_loaded_terms_"+ str(num_clusters)+ "_"+ tweet_set_label + "_bigrams_vb" + str(iteration) +".txt", 'wb') as f:
        for k in range(0,num_clusters):
            #word loadings = cluster_centers * eigenvectors 
            indices = [i for i in np.argsort(word_loadings[k,:])[::-1]]    
            sorted_vocab = [vocab[i] for i in indices]
            f.write("Top words for cluster {}:\n{}\n".format(k, sorted_vocab[:50]))

In [ ]:
with open("top_loaded_terms"+ str(num_clusters)+ "_" +tweet_set_label + "_bigrams_vb" + str(i) +".txt", 'wb') as f:
    for k in range(0,5):
            #word loadings = cluster_centers * eigenvectors 
            indices = [i for i in np.argsort(word_loadings[k,:])[::-1]]    
            sorted_vocab = [vocab[i] for i in indices]
            f.write("Top words for cluster {}:\n{}\n".format(k, sorted_vocab[:50]))

In [ ]:
with open("top_loaded_terms"+ str(num_clusters)+ "_" +tweet_set_label + "_bigrams_vb" + str(i) +".txt", 'rb') as f:
    for line in f:
        print line

In [ ]:
wl = hickle.load('word_loadings5_beer_2012_5cluster_bigrams_vb0.hkl')
wl = hickle.load('word_loadings5_beer_2012_5cluster_bigrams_vb0.hkl')
for i in range(0,5):
    wl = hickle.load('word_loadings5_beer_2012_5cluster_bigrams_vb'+str(i)+'.hkl')
    for k in range(0,5):
        #word loadings = cluster_centers * eigenvectors 
        indices = [i for i in np.argsort(wl[k,:])[::-1]]    
        sorted_vocab = [vocab[i] for i in indices]
        sys.stdout.write("Top words for cluster {}:\n{}\n".format(k, sorted_vocab[:50]))
    sys.stdout.write('\n\n')

In [ ]:
i = 200
with open("top_loaded_terms"+ str(num_clusters)+ "_" +tweet_set_label + "_bigrams_v" + str(i) +".txt", 'wb') as f:
    for k in range(0,50):
        #word loadings = cluster_centers * eigenvectors 
        indices = [i for i in np.argsort(word_loadings[k,:])[::-1]]    
        sorted_vocab = [vocab[i] for i in indices]
        f.write("Top words for cluster {}:\n{}\n".format(k, sorted_vocab[:50]))

In [ ]:
pickle.dump(vocab, open("vocab_"+tweet_set_label + "_BIGRAMS" + ".pkl",'wb'), protocol=2) 

In [ ]:
sys.stdout.write("Printing wordloadings {} to file. \n".format(word_loadings.shape))


In [ ]:
hickle.dump(word_loadings, "word_loadings50_" + tweet_set_label + "_BIGRAMS.hkl", mode='w', compression='gzip')

In [ ]:


hickle.dump(word_loadings, "word_loadings50_" + tweet_set_label + "_BIGRAMS.hkl", mode='w', compression='gzip')


In [ ]:
display(Image(filename='/Users/blehman/Desktop/range1.png') )
print 
print
display(Image(filename='/Users/blehman/Desktop/range2.png') )
print 
print 
display(Image(filename='/Users/blehman/Desktop/range3.png') )

In [ ]:
topic_model_dict['Craft45TopicModel'][0].shape

In [ ]:
np.min(word_loadings)

In [ ]:
np.array([1,1,1,1,0,200])*np.array([[0,0,0,1,0,5],[0,0,0,1,0,5]])

In [ ]:
topic_model_dict = {'FirstTopicEnrichment':( np.load('../enrichments/models/word_loadings.npy')
                                            , np.load('../enrichments/models/vocab.npy'))
                    , 'SecondTopicEnrichment':( np.load('../enrichments/models/word_loadings_v2.npy')
                                               , np.load('../enrichments/models/vocab_v2.npy'))
                    , 'ThirdTopicEnrichment':( pickle.load(open('../enrichments/models/word_loadings_noun','rb'))
                                              , pickle.load(open('../enrichments/models/noun_vectorizer_vocab','rb')))

                    , 'FourthTopicEnrichment':( pickle.load(open('../enrichments/models/word_loadings_noun50','rb'))
                                               , pickle.load(open('../enrichments/models/noun_vectorizer_vocab','rb')))

                    , 'TopicModelB':( hickle.load('../enrichments/models/word_loadings50_.hkl')
                                            , pickle.load(open('../enrichments/models/vocab_.pkl','rb')))

                    , 'TopicModel2': ( hickle.load('../enrichments/models/word_loadings50_bigrams.hkl')
                                              , pickle.load(open('../enrichments/models/vocab_bigrams.pkl')))
                    , 'C45TopicModel': ( hickle.load('../enrichments/models/word_loadings50_bigrams_v1.hkl')
                                            , pickle.load(open('../enrichments/models/vocab_bigrams_v1.pkl')))
                    , 'C45TopicModel_fix': ( hickle.load('../enrichments/models/word_loadings50_bigrams_v2.hkl')
                                                , pickle.load(open('../enrichments/models/vocab_bigrams_v1.pkl')))
                    , 'C42TopicModel': (hickle.load('../enrichments/models/word_loadings50_bigrams_vb4.hkl')
                                            , pickle.load(open('../enrichments/models/vocab_bigrams_vb4.pkl')))
                   }
for topic_model_name,value in topic_model_dict.items():
    word_loadings = value[0]
    vocab = value[1]
    with open(topic_model_name + '_top_wordloadings.txt','wb') as topic_model_file:
        for k in range(0,word_loadings.shape[0]):
            if k == 0:
                topic_model_file.write("{}:\n".format(topic_model_name))
            #word loadings = cluster_centers * eigenvectors 
            indices = [i for i in np.argsort(word_loadings[k,:])[::-1]]    
            sorted_vocab = [vocab[i] for i in indices]
            topic_model_file.write("  Top words for cluster {}:\n{}\n".format(k, sorted_vocab[:50]))
    

In [ ]:
vocab = pickle.load(open('../enrichments/models/vocab_bigrams_vb4.pkl'))


In [ ]:
sys.stdout.write("Printing wordloadings {} to file. \n".format(word_loadings.shape))
with open("top_loaded_terms_"+tweet_set_label + "_bigrams_v" + str(iteration) +".txt", 'wb') as f:
    for k in range(0,50):
        #word loadings = cluster_centers * eigenvectors 
        indices = [i for i in np.argsort(word_loadings[k,:])[::-1]]    
        sorted_vocab = [vocab[i] for i in indices]
        f.write("Top words for cluster {}:\n{}\n".format(k, sorted_vocab[:50]))

In [ ]:
run_here = False
if run_here:
    filePath = '<filePath>'
    if os.path.isfile(filePath):
        testing_data=np.load(filePath)
    else:
        testing_data = [json.loads(line)['body'] for line in get_txt_data(
        np.load('testing.npy')
        , '<filePath>'
        , '<filePath>'
        , True)]


In [ ]:
# Check testing tweets
#print "total testing tweets: {:,}".format(len(testing_data))

We can apply the model to our test data. 

In [ ]:
sample_percentage=0.20
sample_size = int(len(testing_data)*sample_percentage)
label_tweet = zip(result,testing_data[:sample_size])

### Compare distribution of predictions for training vs test

In [ ]:
###
### REVISIT TO build stable clusters [the test set needs rebuilt?]
###

# results from training
display(predictions.pred_cluster.value_counts(normalize=True))

# results from test
display(pd.Series(result).value_counts(normalize=True))

In [ ]:
label_tweet[:50]

### Top loaded terms
We can now explore the clusters to understand their content a bit more. Develop the list of words top loaded into each cluster.

In [ ]:
# create contains to hold vocab    
sorted_vocab_super_set = set()
sorted_vocab_sub_sets = []

# number of words to add to each subcluster
top_n = 20


for k in range(0,5):
    # organize word loading indicies from largest to smallest
    indices=[i for i in np.argsort(word_loadings_test1[k,:])[::-1]]    
    # pull the vocab using the indicies
    sorted_vocab_sub_sets.append(set([reduced_vocab.noun[i] for i in indices][:top_n]))
    
print len(sorted_vocab_sub_sets)

In [ ]:
display(sorted_vocab_sub_sets)

In [ ]:
# check that top_n process worked; each set should have the top_n number of words
for item in sorted_vocab_sub_sets:
    print len(item)

The shortlist of terms above list above might suggest something about the meaning for each cluster.

### Unique terms per topic
This list of terms represents words from the previous top loading list. So basically, we're just removing terms from the shortlist that also appear in other shortlists. This process seems to suggest topics in a human readable form as follows:

- Cluster0: diet conversations
- Cluster1: beer conversations
- Cluster2: mealtime conversations (breakfast specific) 
- Cluster3: mealtime conversations (dinner specific)
- Cluster4: mealtime conversations (lunch specific)


In [ ]:
sorted_vocab_sub_sets

In [ ]:
# set up containers 
sorted_vocab_sub_sets_unique_values = []
set_count = range(len(sorted_vocab_sub_sets))
display(set_count)


def update_set(list_of_lists,superset=set()):
    """
    Flattens a list of lists and appends to the reults to a single set.
    """
    for listN in list_of_lists:
        superset.update(listN)
    return superset

for i in set_count:
    # add new values to the set
    superset = update_set([sorted_vocab_sub_sets[index] for index in set_count if i != index],set())
    # add unique values to subtopic clusterID
    sorted_vocab_sub_sets_unique_values.append( sorted_vocab_sub_sets[i] - superset )
pp.pprint(sorted_vocab_sub_sets_unique_values)

In [ ]:
# count of unique words (w/ top_n) per subcluster
for item in sorted_vocab_sub_sets_unique_values:
    print len(item)

### Topic specific vocabulary
The columns of the word loadings represents the vocabulary terms that are unique to each level  1 topic. Hence, each row counts the weight for the corresponding cluster terms. 

We can iterate through the columns and extract the index of the max word loading values. We consider the largest word loadings for each dimension to determine has the largest affect from that dimension. We will be able to see where each of the 1,109 vocabulary terms has the strongest pull in each cluster.

We can then use the term and corresponding max loaded topic as a means to for a cluster specific vocabulary, which will vectorize tweets from this cluster. We will then apply kmeans again to create subtopics. 

In [ ]:
# (topics, number of terms)
word_loadings_test1.shape


In [ ]:
term_cluster_max={}
with open('<filePath>','wb') as f:
    # Iterate through the dimensions (each column is a word)
    for col in range(word_loadings_test1.shape[1]):
        # find the index of the row with the max word loading
        topicID = np.argmax(word_loadings_test1[:,col])
        # write to file the term's topicID that contains the max weight
        max_val = np.amax(word_loadings_test1[:,col])
        # drop terms that have no weight
        if max_val > 0.0:
            f.write(json.dumps(
                    {
                        reduced_vocab.noun[col]:
                        [
                            topicID # topicID (index)
                            , max_val # max weight (value)
                            , col                                 # column
                        ]
                    })+'\n')
            # build dictionary of the term's topicID that contains the max weight
            term_cluster_max[reduced_vocab.noun[col]] = [
                np.argmax(word_loadings_test1[:,col])
                , np.amax(word_loadings_test1[:,col])
                , col
            ]
        #f.write(json.dumps(term_cluster_max)+'\n')
#sorted_term_cluster_max = sorted(term_cluster_max, key=lambda k: k[0] )
#sorted_term_cluster_max = sorted( term_cluster_max.items(), key=operator.itemgetter(1) )

# sort by the dictionary by the cluster values
sorted_term_cluster_max = sorted( term_cluster_max.items(), key = lambda k: k[1][0] )

In [ ]:
pp.pprint(sorted_term_cluster_max[-10:-1])
print len(sorted_term_cluster_max)
display([item for item in sorted_term_cluster_max if item[0] == u'beer'])

### Visual cluster specific vocabulary
Now that we have each term's strongest topicID, the question remains: how do we explore it? We can plot the number of terms that map strongest to each subtopicID below:

In [ ]:
cluster= {}
# build strong term counts for each cluster
for token,l in sorted_term_cluster_max:
    cluster[l[0]] = cluster.get(l[0],0)+1
 
cluster_df = pd.DataFrame({"Cluster":cluster.keys(),"Term Count":cluster.values()})
display(cluster_df)


The majority of the total 1,109 words map strongest to topic 0 and topic 1 possibly because these are more general conversations.

In [ ]:
cluster_df.plot(x="Cluster",y="Term Count",kind = 'bar',rot=0, title="Term Couns by Cluster ID")

### Build subtopics 
Now we'll take the labeled tweets and use the topic specific vocabulary to vectorize. From this vectorization, we can apply kmeans and find subtopics.  

In [ ]:
cluster_vocab = {ID:[] for ID in cluster.keys() }
display(cluster_vocab)
# We now invert the sorted_term_cluster_max dict to organize the vocabulary by topicID 
for token,l in sorted_term_cluster_max:
    cluster_vocab[ l[0] ].append(token)
cluster_vocab

We'll use these subsets of vocabulary to vectorize the tweets in `X_train` and build the corresponding `sub toipc predictions` with kmeans.

In [ ]:
display(len(predictions))
display(X_train.shape)
display(len(training_data))

labeled_tweets = pd.DataFrame({'clusterID':predictions.pred_cluster, 'tweet':training_data})
labeled_tweets[:5]
labeled_tweets[labeled_tweets.clusterID == 1][:5]

In [ ]:
# Review the lists of lists containing topic specific vocabulary
cluster_vocab[3]

In [ ]:
# set number of clusters
k_subtopics = 5
# create a dataframe containing the labeled tweets
labeled_tweets = pd.DataFrame({'clusterID':predictions.pred_cluster
                               , 'tweet':training_data 
                              })

def create_subtopics_model(cluster_vocab, labeled_tweets, k_subtopics=5):
    """
    """
    centroids_per_cluster = {}
    subtopic_dict = {}
    for cluster,vocab in cluster_vocab.items():
        print 'processing cluster{} using {}'.format(cluster, vocab)
        # build vectorizer function based on the current cluster's vocab
        subtopic_vectorizer = vectorize_(vocab)
        
        # apply the vectorizer to the current cluster's tweets
        X_subtopics = subtopic_vectorizer.fit_transform(labeled_tweets[labeled_tweets.clusterID == cluster].tweet)
        
        # apply lsa
        #lsa = make_pipeline(X_subtopics, Normalizer(copy=False))
        #X_subtopics = lsa.fit_transform(X_subtopics)
        
        # build KMeans function 
        km = KMeans(n_clusters=k_subtopics
                , init='k-means++'
                , max_iter=100
                #, n_init=10
                , verbose=False)
        
        # apply kmeans to the current cluster's tweet set
        km.fit(X_subtopics)
        
        # predcit subtopic clusters
        pred=km.predict(X_subtopics)
        print pred
        # add results to dataframe
        
        subtopic_dict[cluster] = {'tweets':list(labeled_tweets[labeled_tweets.clusterID == cluster].tweet), 'subtopic_label':pred.tolist()}
        #pred_df=pd.DataFrame(pred)
        #pred_df.columns=['pred_cluster']
        
        # add centroids
        centroids_per_cluster[cluster] = km.cluster_centers_
        
        # add inertia (currently not used as an output)
        inertia = km.inertia_
    return centroids_per_cluster , subtopic_dict

subtopic_centroids_dict, labeled_tweets_dict = create_subtopics_model(cluster_vocab, labeled_tweets, k_subtopics=5)

GREAT! So now we have 5 subtopics for each topic. We can see the distributions tweets categorized into the subtopics below. 

In [ ]:
for cluster,d in labeled_tweets_dict.items():
    print cluster, collections.Counter(d['subtopic_label'])

### Top words per sub topic cluster
Let's explore the top words for each sub topic. 

In [ ]:
# save data
with open('subtopic_centroids_dict.json','wb') as f:
    pickle.dump(subtopic_centroids_dict,f)
    np.save(f,subtopic_centroids_dict)
    
with open('cluster_vocab.json','wb') as f:
    pickle.dump(cluster_vocab,f)
    np.save(f,cluster_vocab)
    
with open('labeled_tweets_dict.json','wb') as f:
    pickle.dump(labeled_tweets_dict,f)
    np.save(f,labeled_tweets_dict)

In [ ]:
subtopic_centroids_dict

In [ ]:
labeled_tweets_dict

In [ ]:
len(cluster_vocab[1])

In [ ]:
###
### review alignment w/ original clusters
###

top_n = 50
for cluster,weights in subtopic_centroids_dict.items():
    print '\n Cluster {}: '.format(cluster)
    for i in range(weights.shape[0]):
        print '  Top words for subtopic {}:'.format(i)
        index_vals = np.array(np.argsort(weights[i])[::-1][:top_n])
        print '    {}'.format([ cluster_vocab[cluster][x] for x in index_vals ])

    

### Tweet Analysis by Topic and Subtopic
Let's explore top tweets from each cluster.

In [ ]:
labeled_tweets_dict[0].keys()

In [ ]:
display(labeled_tweets[:10])
#[(k,v) for k,v in labeled_tweets_dict.items()][:1]
subtopic_centroids_dict

for level1Cluster,d in labeled_tweets_dict.items():
    print zip(d['subtopic_label'],d['tweets'])



def label_tweets(vectorizer, testing_data, sample_percentage=0.20):
    """
    Label tweets using word loadings previously derived.
    """
    result = []
    sample_size = int(len(testing_data)*sample_percentage)
    sample_tweets = testing_data[:sample_size]
    for tweet in sample_tweets:
        #sparse_array = vectorizer2.fit_transform([tweet])
        sparse_array = vectorizer.fit_transform([tweet])
        sparse_array_subtraction_abs = np.absolute(sparse_array - word_loadings)
        sparse_array_subtraction_abs_sum = sparse_array_subtraction_abs.sum(axis=1)
        result.append(np.argmin(sparse_array_subtraction_abs_sum))
    return result

#result = label_tweets(vectorizer, testing_data)

In [ ]:
np.argpartition(subtopic_centroids_dict[0][1],np.argmax(subtopic_centroids_dict[0][1]))


In [ ]:
a = np.array([9, 4, 4, 3, 3, 9, 0, 4, 6, 0])

ind1 = np.argsort(a)
display(ind1)


In [ ]:
subtopic_centroids_dict[0]
labeled_tweets_dict.values()

In [ ]:
df = pd.DataFrame({"x":[4,5,6],"y":['A','B','C'], 'bob':[0,0,0]})
df[df['x']==4].bob

In [ ]:
labeled_tweets_dict[:5]

In [ ]:
pd.DataFrame({'clusterID':predictions.pred_cluster, 'tweet':training_data})

In [ ]:
!head term_cluster_max.json

In [ ]:
training_data[:5]

In [ ]:
predictions[:5]

In [ ]:
a = np.array([[1],[1],[1]]).T
b = np.array( [[1,1,1],[2,2,2],[3,3,3],[4,4,4],[5,5,5]] )
a[a>0]

Notes from 2015-09-08:
1.  Add details to 

### csv files for aggregation of all topic models


In [ ]:
with open('daily_2014.csv','rb') as d2014:
    data = d2014.readlines()

In [ ]:
x = pd.read_csv(filepath_or_buffer ='daily_2014.csv'
                ,header=None
                ,names=['TS','tID-label-counter','count','ruleCount','seconds'])
labels = x['tID-label-counter']

In [ ]:
import re
re.findall(r'\d+',
for label in labels:
    tID-label-counter